In [1]:
import os
%pwd

'd:\\end-to-end-ml-project-with-mlflow\\research'

In [2]:
os.chdir("../")
%pwd

'd:\\end-to-end-ml-project-with-mlflow'

In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    criterion: str
    n_estimators: int
    target_column: str

In [4]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForest
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            criterion = params.criterion,
            n_estimators = params.n_estimators,
            target_column = schema.name
            
        )

        return model_trainer_config
    

In [6]:
import os
from mlProject import logger
import joblib
import pandas as pd
from mlProject.components.data_transformation import DataTransformation
from mlProject.config.configuration import ConfigurationManager
from sklearn.ensemble import RandomForestClassifier

In [7]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        # obj1 = ConfigurationManager()
        cfg = ConfigurationManager().get_data_transformation_config()
        # obj2 = DataTransformation(config=cfg)
        train_arr,test_arr = DataTransformation(config=cfg).train_test_splitting()

        x_train,y_train,x_test,y_test = (
            train_arr[:,:-1],
            train_arr[:,-1],
            test_arr[:,:-1],
            test_arr[:,-1]
        )

        # x_train = train_data.drop([self.config.target_column], axis=1)
        # x_test = test_data.drop([self.config.target_column], axis=1)
        # y_train = train_data[[self.config.target_column]]
        # y_test = test_data[[self.config.target_column]]
        
        rfc = RandomForestClassifier(criterion=self.config.criterion, n_estimators=self.config.n_estimators, random_state=42)
        rfc.fit(x_train,y_train)

        joblib.dump(rfc,os.path.join(self.config.root_dir,self.config.model_name))

In [8]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2023-12-05 00:24:16,254: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-05 00:24:16,257: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-05 00:24:16,257: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-05 00:24:16,257: INFO: common: created directory at: artifacts]
[2023-12-05 00:24:16,257: INFO: common: created directory at: artifacts/model_trainer]
[2023-12-05 00:24:16,278: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-05 00:24:16,278: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-05 00:24:16,284: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-05 00:24:16,285: INFO: common: created directory at: artifacts]
[2023-12-05 00:24:16,287: INFO: common: created directory at: artifacts/data_transformation]
[2023-12-05 00:24:16,307: INFO: data_transformation: Splitted the data into test & training data]
[2023-12-05 00:24:16,307: INFO: data_transformation: Shape